---
author: "Ricardo Semião e Castro"
warning: false
message: false
format:
  pdf:
    toc: true
    toc-depth: 3
    number-sections: false
    include-in-header:
      text: |
        \usepackage{sectsty}
        \usepackage{etoolbox}
        \patchcmd{\tableofcontents}{\thispagestyle{plain}}{\thispagestyle{plain}\clearpage}{}{}
        \subsectionfont{\clearpage}
---

# Problem Set 3

## Setup

The files from this problem set can be found in [github.com/ricardo-semiao/task-masters -> quant-macro](https://github.com/ricardo-semiao/task-masters/tree/main/quant-macro).

Importing required libraries:

In [ ]:
import numpy as np
#import polars as pl
#import matplotlib.pyplot as plt
#import seaborn as sns

In this problem set, we will deal with the Huggett (1993) model.

# 1.a) Optimal Decision Rules

## Setup

First, lets define the parameters.

In [ ]:
n = 2 #number of states
states = [1, 0.1] #value of states (unemployed and employed)
probs = np.array([[0.925, 0.075], [0.5, 0.5]]) #transition matrix
avg_end = probs[1, 1] * states[1] + probs[2, 2] * states[2]

beta = 0.96 ** (1 / 6) #discount factor
sigma = 1.5 #CRRA utility function parameter
r = (1 + 0.34) ** (1 / 6) - 1 #interest rate

m = 200 #number of grid points
tol = 1e-6 #tolerance level
max_iter = 1000 #maximum number of iterations

Then, define the grids and other utility objects:

In [ ]:
a = np.linspace(-avg_end, 3 * avg_end, m).reshape(m, 1) #equally spaced row-vector for a

q_bounds = [(beta + tol), 2] #lowest and highest possible q
q = (np.sum(q_bounds)) / 2 #initial guess for q

om = np.ones((1, m)) #auxiliary ones vector

## Functions For Each Step

Now, lets define functions for each step of our algorithm:

- Creating the grid points of consumption and utility.
- Find the value function trough iteration.
- Find the optimal policy functions.
- Create the markov chains for the endogenous transition.
- Update the distribution accordingly.
- Get the final result of the security markets, and adjust `q` for the next iteration.

Lets define functions for each step.

Starting with functions to create consumption grid points, and the related utility:

In [ ]:
def get_c(s, a, q, om, tol):
    c = np.ones((m, 1)) @ np.transpose(a + s) - (q * a) @ om
    c[c < 0] = tol
    return c

def get_u(c, sigma):
    return (c**(1 - sigma) - 1) / (1 - sigma)

The value function:

In [ ]:
def value_function(m, us, beta, om, tol, probs):
    check_vf = np.array([999, 999])
    vs = [np.zeros((m, 1)) for i in range(2)]
    tvs_partial = [0, 0]
    tvs = [np.transpose(np.max(us[i] + beta * vs[i] @ om, axis=0, keepdims=True))
        for i in range(2)]
    
    while np.max(check_vf) > tol:
        vs = tvs
        for i in range(2):
            tvs_partial[i] = us[i] + beta * (probs[i, 0] * vs[0] + probs[i, 1] * vs[1]) * om
            tvs[i] = np.max(tvs_partial[i], axis=0, keepdims=True)
            check_vf[i] = np.linalg.norm(tvs[i] - vs[i]) / np.linalg.norm(vs[i])

    return tvs_partial

The optimal policy function:

In [ ]:
def policy(tvs_partial, a, states, q):
    policy_a = [0, 0]
    policy_c = [0, 0]
    max_vf_ind = [0, 0]

    for i in range(2):
        max_vf_ind[i] = np.argmax(tvs_partial[i], axis=0)
        policy_a[i] = a[max_vf_ind[i], :]
        policy_c[i] = a + states[i] - q * policy_a[i]

    return max_vf_ind, policy_a, policy_c    

The Markov chains:

In [ ]:
def markov(n, m, probs, max_vf_ind):
    mkprobs_aux = [None, None]
    mkprobs = np.zeros((n * m, n * m))
    
    for i in range(2):
        mkprobs_aux[i] = np.zeros((m, m))
        mkprobs_aux[i][:, max_vf_ind[i]] = 1
    
    for i in range(m):
        for j in range(m):
            mkprobs[i*2, 2*j] = probs[0, 0] * mkprobs_aux[0][i, j]
            mkprobs[i*2, 2*j+1] = probs[0, 1] * mkprobs_aux[0][i, j]
            mkprobs[i*2+1, 2*j] = probs[1, 0] * mkprobs_aux[1][i, j]
            mkprobs[i*2+1, 2*j+1] = probs[1, 1] * mkprobs_aux[1][i, j]

    return mkprobs

And finally, the update of the distribution:

In [ ]:
def distribution(n, m, mkprobs, tol):
    dist_prev = np.ones((1, n * m)) / (n * m)
    dist = dist_prev @ mkprobs
    iter_dist = 1
    
    while np.linalg.norm(dist - dist_prev) < tol:
        iter_dist += 1
        if iter_dist == 1000:
            sprintf('!!! Hit maximum amount of iterations on distribution step',)
            break
        dist_prev = dist
        dist = np.matmul(dist, mkprobs)

    return dist

## Algorithm

Now, we can join it all in the main algorithm loop.

In [ ]:
security = 999
iter_total = 0

while abs(security) > tol:
    iter_total += 1
    print(f"Iteration: {iter_total} || sec = {security}, q = {q}")
    
    cs = [get_c(s, a, q, om, tol) for s in states]
    us = [get_u(c, sigma) for c in cs]

    tvs_partial = value_function(m, us, beta, om, tol, probs)

    max_vf_ind, policy_a, policy_c = policy(tvs_partial, a, states, q)

    mkprobs = markov(n, m, probs, max_vf_ind)

    dist = distribution(n, m, mkprobs, tol)

    anext = np.zeros((n * m, 1))
    for i in range(m):
        anext[i * 2, 0] = policy_a[0][i][0]
        anext[i * 2 + 1, 0] = policy_a[1][i][0]
    
    security = dist @ anext
    
    q_bounds[int(security[0][0] > 0)] = q
    q = np.sum(q_bounds) / 2

Iteration: 1 || sec = 999, q = 1.4805
Iteration: 2 || sec = [[-142.16633267]], q = 1.74025
Iteration: 3 || sec = [[-106.]], q = 1.870125
Iteration: 4 || sec = [[-89.]], q = 1.9350625
Iteration: 5 || sec = [[-81.]], q = 1.96753125
Iteration: 6 || sec = [[-77.]], q = 1.983765625
Iteration: 7 || sec = [[-75.]], q = 1.9918828125
Iteration: 8 || sec = [[-74.]], q = 1.99594140625
Iteration: 9 || sec = [[-74.]], q = 1.997970703125
Iteration: 10 || sec = [[-73.]], q = 1.9989853515625
Iteration: 11 || sec = [[-73.]], q = 1.9994926757812501
Iteration: 12 || sec = [[-73.]], q = 1.999746337890625
Iteration: 13 || sec = [[-73.]], q = 1.9998731689453124
Iteration: 14 || sec = [[-73.]], q = 1.9999365844726562


KeyboardInterrupt: 